In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from uuid import uuid4

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels import api as sm

import bhutils
from bhutils import bhutilspy as bh

from api.blpw import BlpQuery
from api.history_plotter import get_expressions_data
from api.tools import get_history_plotter_strings, parse_swap_period
from api.utils import parse_offset
from api.mappings import strategy_map
from api.ezutils import bh, date_to_xl, tsh_to_series, parse_dates, get_irbt_ts

In [3]:
ccy = 'USD'
instrument = 'oswp'
generator = 'usd sofr'
eff_exp = '3m'
tenor_mty = '5yx5y'
strike = 2.15
atm_fixed_date = None
pay_receive = 'pay'
settle = 'pc_lch'
collateral = None
metric = 'spotpremium'

In [ ]:
components = {
    'c1': {'ccy': 'eur', 'instrument': 'irs', }
}

In [4]:
get_irbt_ts(
    ccy=ccy,
    instrument=instrument,
    generator=generator,
    eff_exp=eff_exp,
    tenor_mty=tenor_mty,
    strike=strike,
    atm_fixed_date=atm_fixed_date,
    pay_receive=pay_receive,
    settle=settle,
    collateral=collateral,
    metric=metric,
)

2021-08-04     1.358783
2021-08-05     1.271295
2021-08-06     1.637423
2021-08-09     1.760272
2021-08-10     1.925012
                ...    
2022-07-28    18.361851
2022-07-29    17.211582
2022-08-01    13.992364
2022-08-02    15.428737
2022-08-03    18.914265
Name: USD oswp usd sofr 3mx5yx5y pay @2.15, Length: 261, dtype: float64

In [ ]:
[i for i in dir(bh) if "Remote" in i]

In [ ]:
expressions = [
    
]

In [ ]:
expressions = r[-1].to_dict()
expressions

In [ ]:
%%time
data = get_expressions_data(expressions, history='10y')

In [ ]:
data

In [ ]:
data.plot(backend='plotly', height=700)

In [ ]:
ed5 = get_expressions_data({'ed5': '[s eur 14mx3m]'}, history='10y')

In [ ]:
X = ed5['ed5'].rename('x').to_frame()
y = - data['ptf'].rename('y')

In [ ]:
pd.concat([X, y], axis=1).plot(backend='plotly', height=700)

In [ ]:
def get_df_reg(X, y, diff=False, history=None):
    if diff:
        X = X.diff().dropna()
        y = y.diff().dropna()
    if history:
        X = X.loc[X.index[-1] - parse_offset(history):, :]
        y = y.loc[y.index[-1] - parse_offset(history):]
    df_reg = pd.concat([X, y], axis=1).ffill()
    return df_reg

In [ ]:
df_reg = get_df_reg(
    X, 
    y,
    diff=True,
    history='1y'
)

In [ ]:
lm = LinearRegression(fit_intercept=True)
lm.fit(df_reg.iloc[:, :-1], df_reg.iloc[:, -1]);

In [ ]:
y_pred = pd.Series(lm.predict(df_reg.iloc[:, :-1]), index=df_reg.iloc[:, :-1].index, name='pred')

In [ ]:
f"params: {lm.coef_}, intercept: {lm.intercept_}, r2: {lm.score(df_reg.iloc[:, :-1], df_reg.iloc[:, -1])}"

In [ ]:
((1 / lm.coef_) / 2500) * 1e6

In [ ]:
df_reg.plot.scatter(x='x', y='y', backend='plotly')

In [ ]:
pd.concat([df_reg['y'], y_pred], axis=1).plot.scatter(x='y', y='pred', backend='plotly')

In [ ]:
ols = sm.OLS(df_reg.iloc[:, -1], sm.add_constant(df_reg.iloc[:, :-1]))

In [ ]:
reso = ols.fit()

In [ ]:
reso.summary()

In [ ]:
reso.summary2()